In [1]:
import auxiliary as aux
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

/home/andreafabbricatore/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-13 17:56:56.408174: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747151816.444448 2302599 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747151816.458655 2302599 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747151816.486315 2302599 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than on

In [16]:
# ------------------------------------------------------------------
# Helper: fitness for ONE mutated sentence  (uses aux.compute_metrics)
# ------------------------------------------------------------------
import random

from tqdm import tqdm

# Attack the whole dataset and report new metrics
# ------------------------------------------------------------------
def adversarial_dataset(
    model,
    tokenizer,
    dataset,
    mutation_rate=0.15,
):
    # Cache normal vocab minus specials
    special_ids = set(tokenizer.all_special_ids)
    valid_token_ids = [tid for tid in tokenizer.get_vocab().values()
                       if tid not in special_ids]

    adv_inputs = []
    adv_tokens = []
    adv_labels = []

    for datum in tqdm(dataset, desc="Evolving sentences"):
        new_ids = []
        mutable_ids = [j for i,j in enumerate(datum["input_ids"]) if datum["labels"][i] == 18]
        ids_to_mutate = random.sample(mutable_ids, int(len(mutable_ids) * mutation_rate))
        for i in range(len(datum["input_ids"])):
            if datum["input_ids"][i] in mutable_ids:
                if datum["input_ids"][i] in ids_to_mutate:
                    new_ids.append(random.choice(valid_token_ids))
                    ids_to_mutate.remove(datum["input_ids"][i])
                else:
                    new_ids.append(datum["input_ids"][i])
            else:
                new_ids.append(datum["input_ids"][i])
            
        adv_inputs.append(new_ids)
        adv_tokens.append(tokenizer.convert_ids_to_tokens(new_ids))
        adv_labels.append(datum["labels"])   # labels unchanged

    # Evaluate the whole adversarial corpus
    preds = []
    for ids, datum in tqdm(zip(adv_inputs, dataset),
                           total=len(dataset),
                           desc="Inference on adversarial set"):
        _, p, _, _ = aux.inference(
            model,
            torch.tensor([ids]),
            torch.tensor([datum["attention_mask"]])
        )
        preds.append(p.tolist()[0])

    dataset = dataset.add_column('adv_inputs', adv_inputs)
    dataset = dataset.add_column('adv_tokens', adv_tokens)

    metrics = aux.compute_metrics(preds, adv_labels)
    return metrics, preds, adv_inputs, dataset

In [18]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning

# Suppress only the specific warning from seqeval
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

# 1️⃣ load everything exactly as you did
model      = AutoModelForTokenClassification.from_pretrained('models/distilbert1')
tokenizer  = AutoTokenizer.from_pretrained('models/distilbert1')
train_ds   = aux.json_to_Dataset("data/distilbert_train.json")

# 2️⃣ run the adversarial evolution
adv_metrics, adv_preds, adv_inputs, dataset = adversarial_dataset(
    model,
    tokenizer,
    train_ds.select(range(10)),    # start small for speed
    mutation_rate   = 0.15,
)

print("Metrics on evolved examples:")
for k, v in adv_metrics.items():
    if k != "confusion_matrix":
        print(f"{k:>12}: {v:.4f}")


/home/andreafabbricatore/.local/lib/python3.10/site-packages/transformers/modeling_utils.py:488: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_f

Mutable ids: [3395, 1024, 2177, 24732, 2005, 20247, 2832, 2204, 2851, 1010, 3071, 1010, 1045, 3246, 2023, 4471, 4858, 2017, 2092, 1012, 2004, 2057, 3613, 2256, 20247, 6194, 1010, 1045, 2052, 2066, 2000, 10651, 2017, 2006, 1996, 6745, 8973, 1998, 3145, 2592, 1012, 3531, 2424, 2917, 1996, 17060, 2005, 2256, 9046, 6295, 1024, 1011, 1011, 3116, 2012, 1011, 1011, 3116, 2012, 1011, 1011, 3116, 2012, 1011, 1011, 3116, 2012, 1011]
Ids to mutate: [1011, 1011, 2006, 3116, 2832, 20247, 2005, 2012, 2012, 3613]
Mutate 2005 to 9947
Mutate 20247 to 24665
Mutate 2832 to 3067
Mutate 3613 to 26832
Mutate 2006 to 21305
Mutate 1011 to 22241
Mutate 1011 to 6712
Mutate 3116 to 3511
Mutate 2012 to 3182
Mutate 2012 to 6607
Mutable ids: [1011, 3116, 2012, 1011, 1011, 3116, 2012, 1011, 1011, 3116, 2012, 19045, 2191, 2469, 2000, 14171, 2115, 20283, 11914, 1012, 2256, 2177, 24732, 4132, 2097, 2022, 12550, 2005, 2151, 14409, 1010, 14764, 1010, 2030, 3431, 2000, 1996, 11376, 1012, 2023, 12317, 3921, 2097, 5676, 574

Inference on adversarial set: 100%|██████████| 10/10 [00:03<00:00,  2.97it/s]


Metrics on evolved examples:
   precision: 0.8393
      recall: 0.9216
          f1: 0.8785
    accuracy: 0.9829


In [5]:
aux.write_dataset_to_json(dataset,"test.json")